# Detection of TOXicity in comments in Spanish (DETOXIS 2021)

## SESIÓN 2.2: Clasificación

### Realizado por Álvaro Mazcuñán y Miquel Marín

#### Librerías

Se importan las mismas librerías que se utilizaron en la parte anterior

In [49]:
import pandas as pd
import re
import string
import numpy as np

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.model_selection import train_test_split

from gensim.models import Word2Vec

#### Datos de DETOXIS

A continuación se cargan los datos tal y como se hizo en la anterior entrega

In [40]:
data = pd.read_csv('C:/Users/mique/Documents/CURSO 3 - SEMESTRE B/LENGUAJE NATURAL Y RECUPERACION DE LA INFORMACION/PRACTICAS/DATASET_DETOXIS.csv')
data

,topic,thread_id,comment_id,reply_to,comment_level,comment,argumentation,constructiveness,positive_stance,negative_stance,...,target_group,stereotype,sarcasm,mockery,insult,improper_language,aggressiveness,intolerance,toxicity,toxicity_level
0,CR,0_000,0_002,0_002,1,Pensó: Zumo para restar.,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1
1,CR,0_001,0_003,0_003,1,Como les gusta el afeitado en seco a esta gente.,0,0,0,0,...,1,1,1,1,0,0,0,0,1,1
2,CR,0_002,0_004,0_004,1,"asi me gusta, que se maten entre ellos y en al...",0,0,0,0,...,1,0,0,0,0,0,1,1,1,2
3,CR,0_003,0_005,0_005,1,"Loss mas valientes, los que mejor cortan nuest...",0,0,0,0,...,1,0,1,1,0,0,0,0,1,1
4,CR,0_004,0_006,0_006,1,Costumbres...,0,0,0,0,...,1,1,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3458,MI,20_134,20_164,20_164,1,Ya decía yo que veía menos moros,0,0,0,0,...,1,0,0,1,1,0,0,0,1,1
3459,MI,20_006,20_165,20_008,2,+1. Como lo sabes...,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3460,MI,20_135,20_166,20_166,1,"Seguirán cobrando paguitas en Marruecos,expoli...",0,0,0,0,...,1,1,0,0,0,0,0,1,1,1
3461,MI,20_136,20_167,20_167,1,"pobres, se arriesgan en pateras porque huyen d...",0,0,0,0,...,1,0,0,1,0,0,0,0,1,1


#### Problema del desbalance de clases

Debido a que se van a usar algoritmos de clasificación se tendría que estudiar, para los dos variables de `toxicity` y `toxicity_level`, si dichas clases están balanceadas o no debido a que en problemas de clasificación se suelen encontrar que en el conjunto de datos de entrenamiento una de las clases es minoritaria, es decir, en ella hay muy pocas muestras. Esto puede llegar a afectar a los algoritmos en su proceso de generalización y, en consecuencia, no poder diferenciar una clase de la otra.

In [41]:
data["toxicity"].value_counts()

0    2316
1    1147
Name: toxicity, dtype: int64

In [42]:
data["toxicity_level"].value_counts()

0    2317
1     808
2     269
3      69
Name: toxicity_level, dtype: int64

#### Subset de variables para el análisis

In [43]:
sample_data = data[["comment", "toxicity","toxicity_level"]]
sample_data

,comment,toxicity,toxicity_level
0,Pensó: Zumo para restar.,1,1
1,Como les gusta el afeitado en seco a esta gente.,1,1
2,"asi me gusta, que se maten entre ellos y en al...",1,2
3,"Loss mas valientes, los que mejor cortan nuest...",1,1
4,Costumbres...,1,1
...,...,...,...
3458,Ya decía yo que veía menos moros,1,1
3459,+1. Como lo sabes...,0,0
3460,"Seguirán cobrando paguitas en Marruecos,expoli...",1,1
3461,"pobres, se arriesgan en pateras porque huyen d...",1,1


#### Leer tweets y preprocesado 

In [44]:
def tweet_preprocessing_not_tokenized(tweet):
    tweet = tweet.lower() # Se empieza pasando todos los mensajes a minúsculas
    tweet = re.sub(r"http\S+|www\S+|https\S+", "" ,tweet , flags=re.MULTILINE) # Quitar URLs
    tweet = re.sub(r"\@\w+|\#", "", tweet) # Quitar @ y #
    tweet = re.sub(r"[\U00010000-\U0010ffff]|:\)|:\(|XD|xD|;\)|:,\(|:D|D:", "", tweet) # Quitar emojis y emoticones
    tweet = tweet.translate(str.maketrans('', '', string.punctuation)) # Quitar signos de puntuación
    tokenized_tweets = word_tokenize(tweet)
    filtered_tweets = [word for word in tokenized_tweets if not word in set(stopwords.words('spanish'))] # Quitar stopwords y filtrar
    
    stemming = PorterStemmer() # Inicializamos PorterStemmer para obtener la raíz de cada una de las palabras
    stemmed_tweets = [stemming.stem(word) for word in filtered_tweets]
    lemmatization = WordNetLemmatizer() # Inicializamos el Lemmatizer para obtener los lemas de las palabras
    lemma_tweets = [lemmatization.lemmatize(word, pos='a') for word in stemmed_tweets] 
    return lemma_tweets # NO TOKENIZADO

preprocessing = lambda x: tweet_preprocessing_not_tokenized(x)

In [45]:
sample_data['comment'] = pd.DataFrame(sample_data["comment"].apply(preprocessing))

<ipython-input-45-3c4e5d0f187f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_data['comment'] = pd.DataFrame(sample_data["comment"].apply(preprocessing))


In [46]:
sample_data

,comment,toxicity,toxicity_level
0,"[pensó, zumo, restar]",1,1
1,"[gusta, afeitado, seco, gent]",1,1
2,"[asi, gusta, maten, alta, mar, ma, inmigrant, ...",1,2
3,"[loss, ma, valient, mejor, cortan, cabeza, soc...",1,1
4,[costumbr],1,1
...,...,...,...
3458,"[decía, veía, meno, moro]",1,1
3459,"[1, sabe]",0,0
3460,"[seguirán, cobrando, paguita, marruecosexpolia...",1,1
3461,"[pobr, arriesgan, patera, huyen, guerr, ohhh, ...",1,1
